# Setup

Important: If you're running on a local machine, be sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). This notebook includes only what's necessary to run in Colab.

### Install

In [0]:
# !pip install tensorflow-gpu==1.15
# !pip install -U --pre tensorflow=="2.*"

# Fix for "InvalidArgumentError: TypeError: 'numpy.float64' object cannot be interpreted as an integer"
# Reference: https://stackoverflow.com/questions/59493606/pytorch-and-tensorflow-object-detection-evaluate-object-of-type-class-nump
!pip install numpy==1.17.4

Make sure you have `pycocotools` installed

In [0]:
!pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [0]:
import os
import pathlib


CWD = pathlib.Path.cwd()

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

print()
print(f'Current Working Directory: "{CWD}"')

Fixes for Training section

In [0]:
# Fix for "ImportError: No module named nets"
# Reference: https://github.com/tensorflow/models/issues/1842
!cp -r {CWD}/models/research/slim/nets {CWD}/models/research/object_detection

In [0]:
# Fix for "ValueError: anchor_strides must be a list with the same length as self._box_specs" during training
# Reference: https://stackoverflow.com/questions/53563976/tensorflow-object-detection-api-valueerror-anchor-strides-must-be-a-list-wit
# Modify "/.../models/research/object_detection/anchor_generators/multiple_grid_anchor_generator.py"
!cp {CWD}/multiple_grid_anchor_generator_fix.py {CWD}/models/research/object_detection/anchor_generators/multiple_grid_anchor_generator.py

Compile protobufs and install the object_detection package

In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [0]:
%%bash
cd models/research
pip install .

### Imports

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Import the object detection module.

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loader

In [0]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  MODEL_DIR = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  # Print model checkpoint directory
  print(MODEL_DIR)
  !ls {MODEL_DIR}

  MODEL_DIR = pathlib.Path(MODEL_DIR)/"saved_model"

  # Fix for "TypeError: load() missing 2 required positional arguments: 'tags' and 'export_dir'"
  # Reference: https://stackoverflow.com/questions/58683841/problem-with-running-object-detection-tutorial-typeerror-load-missing-2-requi
  # model = tf.saved_model.load(str(MODEL_DIR))
  model = tf.compat.v2.saved_model.load(str(MODEL_DIR), None)
  model = model.signatures['serving_default']

  return model


def load_custom_model(saved_model_dir):
  # Fix for "TypeError: load() missing 2 required positional arguments: 'tags' and 'export_dir'"
  # Reference: https://stackoverflow.com/questions/58683841/problem-with-running-object-detection-tutorial-typeerror-load-missing-2-requi
  # model = tf.saved_model.load(str(MODEL_DIR))
  model = tf.compat.v2.saved_model.load(str(saved_model_dir), None)
  model = model.signatures['serving_default']

  return model

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [0]:
# Download dataset
!wget -nc https://data.broadinstitute.org/bbbc/BBBC041/malaria.zip
!unzip -u '{CWD}/malaria.zip' -x '__MACOSX/*'

In [0]:
# Parse records
import json
import tensorflow as tf

from object_detection.utils import dataset_util


def get_simplified_image_format(filename: str) -> str:
    extension = filename[-3:]
    
    if extension == 'jpg':
        return 'jpeg'
    else:
        return extension


def get_simplified_class(class_name: str) -> str:
  rbc_list = ['red blood cell', 'difficult']
  infected_list = ['trophozoite', 'schizont', 'ring', 'gametocyte']

  if class_name in rbc_list:
    return 'red blood cell'
  elif class_name in infected_list:
    return 'infected'
  else:
    # Other (i.e. leukocyte)
    return class_name


def create_tf_example(example):
  # TODO(user): Populate the following variables from your example.
  height = example['height'] # Image height
  width = example['width'] # Image width
  filename = example['filename'] # Filename of the image. Empty if image is not from file
  encoded_image_data = example['encoded_image_data'] # Encoded image bytes
  image_format = example['image_format'] # b'jpeg' or b'png'

  xmins = example['xmins'] # List of normalized left x coordinates in bounding box (1 per box)
  xmaxs = example['xmaxs'] # List of normalized right x coordinates in bounding box (1 per box)
  ymins = example['ymins'] # List of normalized top y coordinates in bounding box (1 per box)
  ymaxs = example['ymaxs'] # List of normalized bottom y coordinates in bounding box (1 per box)
  classes_text = example['classes_text'] # List of string class name of bounding box (1 per box)
  classes = example['classes'] # List of integer class id of bounding box (1 per box)

  tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
  }))
  return tf_example

# TODO(user): Write code to read in your dataset to examples variable
TRAINING_JSON_PATH = f'{CWD}/malaria/training.json'
TEST_JSON_PATH = f'{CWD}/malaria/test.json'
dataset_paths = [TRAINING_JSON_PATH, TEST_JSON_PATH]
OUTPUT_PATHS = [f'{CWD}/malaria/training.tfrecord', f'{CWD}/malaria/test.tfrecord']
LABEL_MAP_PATH = f'{CWD}/malaria/classes.pbtxt'
label_map = {}

for i in range(len(dataset_paths)):
  output_path = OUTPUT_PATHS[i]
  # writer = tf.python_io.TFRecordWriter(output_path)
  writer = tf.io.TFRecordWriter(output_path)
  
  with open(dataset_paths[i]) as f:
      data = json.load(f)

  next_index = 1

  for entry in data:
      for obj in entry['objects']:
          if get_simplified_class(obj['category']) not in label_map:
              label_map[get_simplified_class(obj['category'])] = next_index
              next_index += 1
      
      with tf.gfile.GFile(f"{CWD}/malaria{entry['image']['pathname']}", 'rb') as fid:
        encoded_image_data = fid.read()

      dataset_format = {
          'height': entry['image']['shape']['r'],
          'width': entry['image']['shape']['c'],
          'filename': entry['image']['pathname'].encode(),
          'encoded_image_data': encoded_image_data,
          'image_format': get_simplified_image_format(entry['image']['pathname']).encode(),
          'xmins': [obj['bounding_box']['minimum']['c'] / entry['image']['shape']['c'] for obj in entry['objects']],
          'xmaxs': [obj['bounding_box']['maximum']['c'] / entry['image']['shape']['c'] for obj in entry['objects']],
          'ymins': [obj['bounding_box']['minimum']['r'] / entry['image']['shape']['r'] for obj in entry['objects']],
          'ymaxs': [obj['bounding_box']['maximum']['r'] / entry['image']['shape']['r'] for obj in entry['objects']],
          'classes_text': [get_simplified_class(obj['category']).encode() for obj in entry['objects']],
          'classes': [label_map[get_simplified_class(obj['category'])] for obj in entry['objects']]
      }
      
      tf_example = create_tf_example(dataset_format)
      writer.write(tf_example.SerializeToString())

  writer.close()

# Create label map
with open(LABEL_MAP_PATH, 'w') as f:
  first_label = True

  for label in label_map:
    f.write('item {\n') if first_label else f.write('\nitem {\n')
    f.write(f'  id: {label_map[label]}\n')
    f.write(f'  name: "{label}"\n')
    f.write('}\n')

    first_label = False

print('Verify "ssd.config" matches the following:')
print(f'train_/eval_input_reader => label_map_path: "{LABEL_MAP_PATH}"')
print(f'train_input_reader => input_path: "{OUTPUT_PATHS[0]}"')
print(f'eval_input_reader => input_path: "{OUTPUT_PATHS[1]}"')

In [0]:
# List of the strings that is used to add correct label for each box.
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH, use_display_name=True)

category_index

In [0]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path(f'{CWD}/malaria/images')
TEST_IMAGE_PATHS = []
EXTENSIONS = ['*.jpg', '*.png']

for ext in EXTENSIONS:
  TEST_IMAGE_PATHS.extend(list(PATH_TO_TEST_IMAGES_DIR.glob(ext)))

TEST_IMAGE_PATHS.sort()

# Training

In [0]:
model_name = 'ssd_mobilenet_v2_coco_2018_03_29'
detection_model = load_model(model_name)

# ssd.config file => fine_tune_checkpoint: "/root/.keras/datasets/ssd_mobilenet_v2_coco_2018_03_29/model.ckpt"

In [0]:
pipeline_fname = f'{CWD}/ssd.config'
model_dir = f'{CWD}/training'
num_steps = 10000
num_eval_steps = 10000

In [0]:
!python {CWD}/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

# Detection

Load an object detection model:

In [0]:
# model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
# detection_model = load_model(model_name)
detection_model = load_custom_model(model_dir)

Check the model's input signature, it expects a batch of 3-color images of type uint8: 

In [0]:
print(detection_model.inputs)

And retuns several outputs:

In [0]:
detection_model.output_dtypes

In [0]:
detection_model.output_shapes

Add a wrapper function to call the model, and cleanup the outputs:

In [0]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)
  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

Run it on each test image and show the results:

In [0]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  # print(output_dict)
  display(Image.fromarray(image_np))

In [0]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)
  break

## Instance Segmentation

In [0]:
model_name = "mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28"
masking_model = load_model("mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28")

The instance segmentation model includes a `detection_masks` output:

In [0]:
masking_model.output_shapes

In [0]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(masking_model, image_path)